## Homework

In [141]:
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [143]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs, make_circles, make_classification, load_iris, load_digits
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree, metrics, model_selection
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import Counter

In [144]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [145]:
#X = df.drop(columns=["price"])
#y = df.price

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [146]:
df.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [147]:
drop_columns = [
    'id',           
    'timestamp']

cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]

num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [151]:
X = df.drop(columns=["price"])
y = df.price

X = X.drop(columns=drop_columns)
X = X.drop(columns=cat_columns)

X.fillna(0.0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtr = DecisionTreeRegressor(max_depth=2, random_state=42)
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)

Проверьте качество на отложенной выборке.

In [152]:
metrics.mean_squared_error(y_pred, y_test) ** 0.5

3560515.64355517

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [153]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

<ipython-input-153-ee3edee7ad76>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


In [154]:
df.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2,15,33,1,12,75,10,15318960,1122,112
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,0,0,13,1,0,6,1,6080000,327,68
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,1,8,18,0,1,52,0,17000000,752,192
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,1,3,0,2,8,2,990000,262,69
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,0,5,8,0,1,34,5,7900000,711,214


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [155]:
df.fillna(0.0, inplace=True)

df["floor_per_max_floor"] = df.floor / (df.max_floor + 0.0001)
df["kitch_per_flat_sq"] = df.kitch_sq / (df.full_sq + 0.0001)
df["month"] = df.timestamp.dt.month
df["day_of_week"] = df.timestamp.dt.dayofweek

df.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt,floor_per_max_floor,kitch_per_flat_sq,month,day_of_week
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12,75,10,15318960,1122,112,0.999900,0.999900,12,4
1,1,2012-10-04,64,64.0,16.0,0.0,0.0,0.0,0.0,0.0,...,0,6,1,6080000,327,68,160000.000000,0.000000,10,3
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,1,52,0,17000000,752,192,0.529409,0.120482,2,2
3,3,2012-07-26,71,49.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2,8,2,990000,262,69,20000.000000,0.000000,7,3
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,1,34,5,7900000,711,214,0.999989,0.100000,10,2


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [156]:
X = df.drop(columns=["price"])
y = df.price

X = X.drop(columns=drop_columns)
X = X.drop(columns=cat_columns)

X.fillna(0.0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtr = DecisionTreeRegressor(max_depth=2, random_state=42)
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)

metrics.mean_squared_error(y_pred, y_test) ** 0.5

3560515.64355517

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [157]:
dtr = DecisionTreeRegressor(max_depth=5, random_state=42)
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)

metrics.mean_squared_error(y_pred, y_test) ** 0.5

3110006.438354547

In [114]:
rfr = RandomForestRegressor(max_depth=5, random_state=42)
rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)

metrics.mean_squared_error(y_pred, y_test) ** 0.5

2855114.8618917847

In [118]:
#!pip install catboost

from catboost import CatBoostRegressor

In [131]:
cbr = CatBoostRegressor(max_depth=5, random_state=42)
cbr.fit(X_train, y_train)

y_pred = cbr.predict(X_test)

metrics.mean_squared_error(y_pred, y_test) ** 0.5

Learning rate set to 0.063934
0:	learn: 4561372.3127607	total: 28.9ms	remaining: 28.8s
1:	learn: 4421491.1457232	total: 56.3ms	remaining: 28.1s
2:	learn: 4292259.5293483	total: 81.7ms	remaining: 27.2s
3:	learn: 4169878.5140415	total: 106ms	remaining: 26.5s
4:	learn: 4066263.9151396	total: 132ms	remaining: 26.2s
5:	learn: 3961699.6739452	total: 157ms	remaining: 26.1s
6:	learn: 3868904.1640508	total: 184ms	remaining: 26.1s
7:	learn: 3781386.3282032	total: 221ms	remaining: 27.5s
8:	learn: 3696385.0264074	total: 248ms	remaining: 27.3s
9:	learn: 3617951.5417367	total: 273ms	remaining: 27s
10:	learn: 3550832.2508046	total: 298ms	remaining: 26.8s
11:	learn: 3489768.4212045	total: 323ms	remaining: 26.6s
12:	learn: 3436697.7019314	total: 347ms	remaining: 26.3s
13:	learn: 3379656.8506471	total: 371ms	remaining: 26.1s
14:	learn: 3328151.6680297	total: 410ms	remaining: 26.9s
15:	learn: 3284861.7035936	total: 437ms	remaining: 26.9s
16:	learn: 3243975.6492676	total: 463ms	remaining: 26.8s
17:	learn:

2547653.905549915

In [ ]:
Лучший результат показал CatBoostRegressor

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [125]:
df.shape

(20000, 298)

In [126]:
df_owner = df[df.product_type == "OwnerOccupier"]
df_invest = df[df.product_type == "Investment"]

((7160, 298), (12840, 298))

In [132]:
X_owner = df_owner.drop(columns=["price"])
y_owner = df_owner.price

X_owner = X_owner.drop(columns=drop_columns)
X_owner = X_owner.drop(columns=cat_columns)

X_owner.fillna(0.0, inplace=True)

X_owner_train, X_owner_test, y_owner_train, y_owner_test = train_test_split(X_owner, y_owner, test_size=0.2, random_state=42)

cbr = CatBoostRegressor(max_depth=5, random_state=42)
cbr.fit(X_owner_train, y_owner_train)

y_owner_pred = cbr.predict(X_owner_test)

mse_owner = metrics.mean_squared_error(y_owner_pred, y_owner_test)

Learning rate set to 0.053251
0:	learn: 4044415.9284193	total: 18.4ms	remaining: 18.4s
1:	learn: 3906607.0480434	total: 33.5ms	remaining: 16.7s
2:	learn: 3765541.2021556	total: 48.5ms	remaining: 16.1s
3:	learn: 3650196.1546320	total: 63.5ms	remaining: 15.8s
4:	learn: 3534765.4054358	total: 79ms	remaining: 15.7s
5:	learn: 3422484.2223152	total: 94.7ms	remaining: 15.7s
6:	learn: 3319890.4347273	total: 110ms	remaining: 15.6s
7:	learn: 3226707.8191487	total: 125ms	remaining: 15.5s
8:	learn: 3128009.5221513	total: 140ms	remaining: 15.4s
9:	learn: 3046658.1056705	total: 156ms	remaining: 15.4s
10:	learn: 2970698.0835468	total: 171ms	remaining: 15.4s
11:	learn: 2894996.8470891	total: 199ms	remaining: 16.4s
12:	learn: 2819877.9345148	total: 215ms	remaining: 16.3s
13:	learn: 2748147.6299394	total: 231ms	remaining: 16.3s
14:	learn: 2681859.3797195	total: 246ms	remaining: 16.1s
15:	learn: 2620342.0051531	total: 261ms	remaining: 16.1s
16:	learn: 2555910.6751925	total: 276ms	remaining: 16s
17:	learn

In [133]:
X_invest = df_invest.drop(columns=["price"])
y_invest = df_invest.price

X_invest = X_invest.drop(columns=drop_columns)
X_invest = X_invest.drop(columns=cat_columns)

X_invest.fillna(0.0, inplace=True)

X_invest_train, X_invest_test, y_invest_train, y_invest_test = train_test_split(X_invest, y_invest, test_size=0.2, random_state=42)

cbr = CatBoostRegressor(max_depth=5, random_state=42)
cbr.fit(X_invest_train, y_invest_train)

y_invest_pred = cbr.predict(X_invest_test)

mse_invest = metrics.mean_squared_error(y_invest_pred, y_invest_test)

Learning rate set to 0.059085
0:	learn: 4775602.5615370	total: 239ms	remaining: 3m 58s
1:	learn: 4651007.3754877	total: 465ms	remaining: 3m 52s
2:	learn: 4537612.3446333	total: 675ms	remaining: 3m 44s
3:	learn: 4432643.4510742	total: 923ms	remaining: 3m 49s
4:	learn: 4329591.4753873	total: 1.16s	remaining: 3m 50s
5:	learn: 4239326.7939757	total: 1.37s	remaining: 3m 46s
6:	learn: 4156684.3809746	total: 1.58s	remaining: 3m 43s
7:	learn: 4081781.3551659	total: 1.75s	remaining: 3m 37s
8:	learn: 4006267.9995292	total: 1.97s	remaining: 3m 37s
9:	learn: 3938075.9626260	total: 2.17s	remaining: 3m 35s
10:	learn: 3875157.8899390	total: 2.4s	remaining: 3m 35s
11:	learn: 3815178.5074825	total: 2.58s	remaining: 3m 32s
12:	learn: 3759235.0365743	total: 2.81s	remaining: 3m 33s
13:	learn: 3701586.2411488	total: 3s	remaining: 3m 31s
14:	learn: 3655003.0949145	total: 3.23s	remaining: 3m 31s
15:	learn: 3612439.7641647	total: 3.4s	remaining: 3m 29s
16:	learn: 3571271.3250755	total: 3.62s	remaining: 3m 29s

In [134]:
(mse_invest + mse_owner) ** 0.5

3155609.9562797705

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
cbr = CatBoostRegressor(max_depth=5, random_state=42)
cbr.fit(X_invest_train, y_invest_train)

y_invest_pred_cbr = cbr.predict(X_invest_test)


hr = HuberRegressor(max_iter=50)
hr.fit(X_invest_train, y_invest_train)

y_invest_pred_hr = cbr.predict(X_invest_test)


w_1 = 0.3
w_2 = 1 - w_1


y_pred_ens = y_invest_pred_cbr * w_1 + y_invest_pred_hr * w_2


metrics.mean_squared_error(y_pred_ens, y_invest_test) ** 0.5

In [ ]:
metrics.mean_squared_error(y_pred_ens, y_invest_test) ** 0.5